## Init

In [11]:
import ccxt
import datetime
import pandas as pd
import pystore
import os
import bt
# storage_dir = os.path.abspath(os.path.curdir + '/storage')
# kucoin = ccxt.kucoin({'apiKey': '', 'secret': '', 'password': ''})
# balance = kucoin.fetchBalance()
# all_symbols = kucoin.symbols
# all_usdt_symbols = list(filter(lambda x: x.endswith("USDT"), all_symbols))

# current_symbols = ["ETH","BTC","MATIC","CRO","FIL","UNI","BAT","LINK","GRT","AAVE","COMP","SNX"]
# Not available in my portfolio: NU, ROOK, ARMOR, BADGER
storage_dir = '/Volumes/CAPA/.storage'
data = {}
pystore.set_path(storage_dir)
store = pystore.store('capa_store')
collection = store.collection('Crypto.Candles')
collections = store.list_collections()
items = collection.list_items()
available_symbols = items

for symbol in current_symbols:
    data[symbol] = collection.item(symbol + '/Day').to_pandas()
    data[symbol]['timestamp'] = data[symbol].index
closing_prices = merge_timeseries(data, 'close')
import csv
import os
from os.path import dirname, abspath
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import pystore
import ccxt
import datetime
import pandas as pd

from messari import Messari
columns = {}
messari = Messari('')
# all_assets = messari.get_all_assets()
metrics = messari.list_asset_timeseries_metric_ids()['data']['metrics']
free = list(filter(lambda x: 'role_restriction' not in x, metrics))
pro = list(filter(lambda x: 'role_restriction' in x, metrics))
for x in free:
    start = ','
    values = start.join(list(x['values_schema'].keys()))
    columns[x['metric_id']] = values
all_metrics = list(columns.keys())
# def fetch_data(symbol):
#     file = pd.read_csv(data_dir+symbol)
#     # file['time'] = pd.to_datetime(file["time"], unit="ms")
#     # file.set_index('time',drop=False, inplace=True)
#     # if "Unnamed: 0" in file.columns:
#     #     del file["Unnamed: 0"]
#     return file


# def get_weights_equal():
#     weights = {}
#     for symbol in current_symbols:
#         weights[symbol] = 1/len(current_symbols)
#     return weights

[timestamp
2017-10-19     294.994204
2017-10-20     274.673215
2017-10-23     298.739969
2017-10-24     311.913000
2017-10-25     297.000000
                 ...     
2021-06-01    2634.530000
2021-06-02    2707.580000
2021-06-03    2857.030000
2021-06-04    2688.550000
2021-06-05    2616.270000
Name: ETH, Length: 1310, dtype: float64, timestamp
2017-10-19     5137.927269
2017-10-20     5698.297439
2017-10-21     5985.000000
2017-10-22     6500.000000
2017-10-23     6150.000000
                  ...     
2021-06-01    36689.400000
2021-06-02    37589.000000
2021-06-03    39249.900000
2021-06-04    36829.300000
2021-06-05    35770.800000
Name: BTC, Length: 1319, dtype: float64, timestamp
2021-05-13    1.0541
2021-05-14    1.4695
2021-05-15    1.5926
2021-05-16    1.7236
2021-05-17    1.7283
2021-05-18    2.4421
2021-05-19    1.6191
2021-05-20    1.8302
2021-05-21    1.4842
2021-05-22    1.1953
2021-05-23    1.0729
2021-05-24    1.7460
2021-05-25    1.9346
2021-05-26    2.2085
2021-05-27

## Helper Functions and Utils

In [15]:
def merge_timeseries(datasets, label):
    dfs = []
    for asset, df in datasets.items():
        dfs.append(df[label].rename(asset))
    merged = pd.concat(dfs, axis=1)
    merged.ffill(inplace=True)
    # merged.fillna(0, inplace=True)
    return merged

def change_percent(frequency, dataset, append_column=True):
    # Get price value for each offset
    value = dataset["close"].pct_change(freq=frequency)
    if append_column:
        dataset[frequency+"_percent_change"] = value
    return value

def scaled_close(dataset, append_column=True):
    # Get price value for each offset
    value = dataset["close"] / max(dataset["close"])
    if append_column:
        dataset["scaled_close"] = value
    return value

## Load Data from Pystore using Widget

In [16]:
symbols = widgets.Dropdown(
    options=sorted(available_symbols),
    description='Symbol:',
    disabled=False
)
display(symbols)

add_symbol_button = widgets.Button(description="Add Symbol")
remove_symbol_button = widgets.Button(description="Remove Symbol")

def add_symbol(b):
    current_symbols.append(symbols.value)
    data[symbols.value] = collection.item(symbols.value + '/USDT').to_pandas()
    # scaled_close(data[symbols.value])
    with output:
        print("Added "+ symbols.value, 'Current Assets: '+ ", ".join(current_symbols))

def remove_symbol(b):
    current_symbols.remove(symbols.value)
    del data[symbols.value]
    weights_equal = get_weights_equal()
    with output:
        print("Removed "+ symbols.value, 'Current Assets: '+ ", ".join(current_symbols))

add_symbol_button.on_click(add_symbol)
remove_symbol_button.on_click(remove_symbol)
display(add_symbol_button, remove_symbol_button)

Dropdown(description='Symbol:', options=('1INCH', 'AAVE', 'ABBC', 'ACE', 'ACOIN', 'ADA', 'ADA3L', 'ADA3S', 'AK…

Button(description='Add Symbol', style=ButtonStyle())

Button(description='Remove Symbol', style=ButtonStyle())

## Equal Weighted Rebalancing - Weekly, Monthly, Quarterly, Yearly

In [17]:
# s0 = bt.Strategy('Hourly Equal', [
#     bt.algos.RunEveryNPeriods(60),
#     bt.algos.SelectHasData(),
#     bt.algos.WeighEqually(),
#     bt.algos.Rebalance()
#     ]
# )

s1 = bt.Strategy('Weekly Equal', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s2 = bt.Strategy('Monthly Equal', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s3 = bt.Strategy('Quarterly Equal', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s4 = bt.Strategy('Yearly Equal', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_data['LINK'] = data["LINK"]
bench_closing_prices = merge_timeseries(bench_data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
# test0 = bt.Backtest(s0, closing_prices)
test1 = bt.Backtest(s1, closing_prices)
test2 = bt.Backtest(s2, closing_prices)
test3 = bt.Backtest(s3, closing_prices)
test4 = bt.Backtest(s4, closing_prices)
res1 = bt.run(test1,test2,test3,test4, test_bench)
res1.display()

Stat                 Weekly Equal    Monthly Equal    Quarterly Equal    Yearly Equal    Long BTC/ETH Equal
-------------------  --------------  ---------------  -----------------  --------------  --------------------
Start                2017-10-18      2017-10-18       2017-10-18         2017-10-18      2017-10-18
End                  2021-06-05      2021-06-05       2021-06-05         2021-06-05      2021-06-05
Risk-free rate       0.00%           0.00%            0.00%              0.00%           0.00%

Total Return         1169.88%        1394.55%         1768.03%           3480.81%        690.36%
Daily Sharpe         1.02            1.05             1.10               1.25            0.92
Daily Sortino        1.65            1.72             1.82               2.09            1.47
CAGR                 101.39%         110.63%          123.98%            167.95%         76.73%
Max Drawdown         -87.93%         -87.88%          -85.91%            -87.33%         -87.80%
Calmar R

## Inverse Volatility Weighted Rebalancing - Weekly, Monthly, Quarterly, Yearly

In [18]:
# s00 = bt.Strategy('Hourly Inv Vol', [
#     bt.algos.RunEveryNPeriods(60),
#     bt.algos.SelectHasData(),
#     bt.algos.WeighInvVol(),
#     bt.algos.Rebalance()
#     ]
# )

s5 = bt.Strategy('Weekly Inv Vol', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s6 = bt.Strategy('Monthly Inv Vol', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s7 = bt.Strategy('Quarterly Inv Vol', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s8 = bt.Strategy('Yearly Inv Vol', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test5 = bt.Backtest(s5, closing_prices)
test6 = bt.Backtest(s6, closing_prices)
test7 = bt.Backtest(s7, closing_prices)
test8 = bt.Backtest(s8, closing_prices)
res2 = bt.run(test5,test6,test7,test8, test_bench)
res2.display()

Stat                 Weekly Inv Vol    Monthly Inv Vol    Quarterly Inv Vol    Yearly Inv Vol    Long BTC/ETH Equal
-------------------  ----------------  -----------------  -------------------  ----------------  --------------------
Start                2017-10-18        2017-10-18         2017-10-18           2017-10-18        2017-10-18
End                  2021-06-05        2021-06-05         2021-06-05           2021-06-05        2021-06-05
Risk-free rate       0.00%             0.00%              0.00%                0.00%             0.00%

Total Return         991.87%           982.81%            857.43%              1707.14%          690.36%
Daily Sharpe         1.00              1.00               0.97                 1.12              0.92
Daily Sortino        1.58              1.58               1.52                 1.80              1.47
CAGR                 93.18%            92.74%             86.32%               121.94%           76.73%
Max Drawdown         -86.05%     

## Mean Variance Weighted Rebalancing - Weekly, Monthly, Quarterly, Yearly

In [19]:
s000 = bt.Strategy('Hourly Mean Var', [
    bt.algos.RunEveryNPeriods(60),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s9 = bt.Strategy('Weekly Mean Var', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s10 = bt.Strategy('Monthly Mean Var', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s11 = bt.Strategy('Quarterly Mean Var', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s12 = bt.Strategy('Yearly Mean Var', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test9 = bt.Backtest(s9, closing_prices)
test10 = bt.Backtest(s10, closing_prices)
test11 = bt.Backtest(s11, closing_prices)
test12 = bt.Backtest(s12, closing_prices)
res3 = bt.run(test9,test10,test11,test12, test_bench)
res3.display()

Stat                 Weekly Mean Var    Monthly Mean Var    Quarterly Mean Var    Yearly Mean Var    Long BTC/ETH Equal
-------------------  -----------------  ------------------  --------------------  -----------------  --------------------
Start                2017-10-18         2017-10-18          2017-10-18            2017-10-18         2017-10-18
End                  2021-06-05         2021-06-05          2021-06-05            2021-06-05         2021-06-05
Risk-free rate       0.00%              0.00%               0.00%                 0.00%              0.00%

Total Return         507.93%            791.24%             614.05%               7447.29%           690.36%
Daily Sharpe         0.84               0.92                0.88                  1.27               0.92
Daily Sortino        1.36               1.55                1.57                  2.42               1.47
CAGR                 64.40%             82.67%              71.85%                229.04%            76.7

## ERC Weighted Rebalancing - Weekly, Monthly, Quarterly, Yearly

In [20]:
s00 = bt.Strategy('Hourly ERC', [
    bt.algos.RunEveryNPeriods(60),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

s13 = bt.Strategy('Weekly ERC', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

s14 = bt.Strategy('Monthly ERC', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

s15 = bt.Strategy('Quarterly ERC', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

s16 = bt.Strategy('Yearly ERC', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(bench_data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test13 = bt.Backtest(s13, closing_prices)
test14 = bt.Backtest(s14, closing_prices)
test15 = bt.Backtest(s15, closing_prices)
test16 = bt.Backtest(s16, closing_prices)
res4 = bt.run(test13,test14,test15,test16, test_bench)
res4.display()

Stat                 Weekly ERC    Monthly ERC    Quarterly ERC    Yearly ERC    Long BTC/ETH Equal
-------------------  ------------  -------------  ---------------  ------------  --------------------
Start                2017-10-18    2017-10-18     2017-10-18       2017-10-18    2017-10-18
End                  2021-06-05    2021-06-05     2021-06-05       2021-06-05    2021-06-05
Risk-free rate       0.00%         0.00%          0.00%            0.00%         0.00%

Total Return         936.29%       1036.90%       1087.35%         2597.98%      690.36%
Daily Sharpe         0.98          1.00           1.01             1.20          0.92
Daily Sortino        1.56          1.60           1.62             1.97          1.47
CAGR                 90.42%        95.34%         97.69%           147.85%       76.73%
Max Drawdown         -86.82%       -86.62%        -85.20%          -87.33%       -87.80%
Calmar Ratio         1.04          1.10           1.15             1.69          0.87

M

## Weekly Rebalancing - Equal, Inv Vol, Mean Var, ERC

In [4]:
s9 = bt.Strategy('Weekly Equal', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s10 = bt.Strategy('Weekly Inv Vol', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s11 = bt.Strategy('Weekly Mean Var', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s12 = bt.Strategy('Weekly ERC', [
    bt.algos.RunWeekly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test9 = bt.Backtest(s9, closing_prices)
test10 = bt.Backtest(s10, closing_prices)
test11 = bt.Backtest(s11, closing_prices)
test12 = bt.Backtest(s12, closing_prices)
res3 = bt.run(test9,test10,test11,test12, test_bench)
res3.display()

NameError: name 'merge_timeseries' is not defined

## Monthly Rebalancing - Equal, Inv Vol, Mean Var, ERC

In [ ]:
s9 = bt.Strategy('Monthly Equal', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s10 = bt.Strategy('Monthly Inv Vol', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s11 = bt.Strategy('Monthly Mean Var', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s12 = bt.Strategy('Monthly ERC', [
    bt.algos.RunMonthly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test9 = bt.Backtest(s9, closing_prices)
test10 = bt.Backtest(s10, closing_prices)
test11 = bt.Backtest(s11, closing_prices)
test12 = bt.Backtest(s12, closing_prices)
res3 = bt.run(test9,test10,test11,test12, test_bench)
res3.display()

## Quarterly Rebalancing - Equal, Inv Vol, Mean Var, ERC

In [ ]:
s9 = bt.Strategy('Quarterly Equal', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s10 = bt.Strategy('Quarterly Inv Vol', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s11 = bt.Strategy('Quarterly Mean Var', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s12 = bt.Strategy('Quarterly ERC', [
    bt.algos.RunQuarterly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test9 = bt.Backtest(s9, closing_prices)
test10 = bt.Backtest(s10, closing_prices)
test11 = bt.Backtest(s11, closing_prices)
test12 = bt.Backtest(s12, closing_prices)
res3 = bt.run(test9,test10,test11,test12, test_bench)
res3.display()

## Yearly Rebalancing - Equal, Inv Vol, Mean Var, ERC

In [ ]:
s9 = bt.Strategy('Yearly Equal', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)

s10 = bt.Strategy('Yearly Inv Vol', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighInvVol(),
    bt.algos.Rebalance()
    ]
)

s11 = bt.Strategy('Yearly Mean Var', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighMeanVar(),
    bt.algos.Rebalance()
    ]
)

s12 = bt.Strategy('Yearly ERC', [
    bt.algos.RunYearly(),
    bt.algos.SelectHasData(),
    bt.algos.WeighERC(),
    bt.algos.Rebalance()
    ]
)

bench = bt.Strategy("Long BTC/ETH Equal", [
    bt.algos.RunOnce(),
    bt.algos.SelectAll(),
    bt.algos.WeighEqually(),
    bt.algos.Rebalance()
    ]
)
bench_data = {}
bench_data['BTC'] = data["BTC"]
bench_data['ETH'] = data["ETH"]
bench_closing_prices = merge_timeseries(bench_data, 'close')
test_bench = bt.Backtest(bench, bench_closing_prices)
test9 = bt.Backtest(s9, closing_prices)
test10 = bt.Backtest(s10, closing_prices)
test11 = bt.Backtest(s11, closing_prices)
test12 = bt.Backtest(s12, closing_prices)
res3 = bt.run(test9,test10,test11,test12, test_bench)
print(res3.stats.index)
res3.stats.loc['total_return']

In [21]:
allresults = bt.run(test1,test2,test3,test4,test5,test6,test7,test8,test9,test10,test11,test12,test13,test14,test15,test16, test_bench)


In [23]:
print(allresults.stats.index)
allresults.stats.loc[['total_return', 'monthly_sharpe', 'max_drawdown', 'cagr', 'ytd', 'daily_sharpe','monthly_vol','avg_drawdown', 'avg_drawdown_days', 'avg_up_month', 'avg_down_month'], :].T.sort_values('total_return', ascending=False)

Index(['start', 'end', 'rf', 'total_return', 'cagr', 'max_drawdown', 'calmar',
       'mtd', 'three_month', 'six_month', 'ytd', 'one_year', 'three_year',
       'five_year', 'ten_year', 'incep', 'daily_sharpe', 'daily_sortino',
       'daily_mean', 'daily_vol', 'daily_skew', 'daily_kurt', 'best_day',
       'worst_day', 'monthly_sharpe', 'monthly_sortino', 'monthly_mean',
       'monthly_vol', 'monthly_skew', 'monthly_kurt', 'best_month',
       'worst_month', 'yearly_sharpe', 'yearly_sortino', 'yearly_mean',
       'yearly_vol', 'yearly_skew', 'yearly_kurt', 'best_year', 'worst_year',
       'avg_drawdown', 'avg_drawdown_days', 'avg_up_month', 'avg_down_month',
       'win_year_perc', 'twelve_month_win_perc'],
      dtype='object')


,total_return,monthly_sharpe,max_drawdown,cagr,ytd,daily_sharpe,monthly_vol,avg_drawdown,avg_drawdown_days,avg_up_month,avg_down_month
Yearly Mean Var,74.472904,1.193252,-0.878896,2.290351,1.090732,1.270285,2.040961,-0.154894,22.509804,0.607064,-0.166027
Yearly Equal,34.808073,1.26536,-0.873337,1.679464,2.100081,1.245878,1.322398,-0.113127,22.78,0.403019,-0.149236
Yearly ERC,25.979762,1.249074,-0.873337,1.478466,1.885569,1.199576,1.215637,-0.103384,22.333333,0.358297,-0.15158
Quarterly Equal,17.680305,1.118648,-0.859057,1.239783,2.018029,1.100616,1.2998,-0.101563,22.86,0.355734,-0.16031
Yearly Inv Vol,17.071401,1.195646,-0.869457,1.21943,1.836106,1.121919,1.103666,-0.104732,23.791667,0.311023,-0.154582
Monthly Equal,13.945514,1.094026,-0.878849,1.106308,2.463842,1.052803,1.259287,-0.112654,29.717949,0.349923,-0.16733
Weekly Equal,11.698753,1.078685,-0.879346,1.013881,1.990664,1.016428,1.185446,-0.105446,31.567568,0.319145,-0.173157
Quarterly ERC,10.873529,1.083119,-0.852013,0.97695,1.892901,1.013869,1.105865,-0.094376,28.925,0.309706,-0.152054
Monthly ERC,10.369024,1.077597,-0.86619,0.953447,2.447726,1.004395,1.102237,-0.096123,30.447368,0.310638,-0.155008
Weekly Inv Vol,9.918726,1.087494,-0.86054,0.931822,2.25342,1.001658,1.03503,-0.109615,33.257143,0.284142,-0.156653


Scheduled Tasks:

Hourly:
Fetch minute candles and append (all symbols, since last day in column)
Fetch balances (all exchanges?)

Daily:
fetch all symbols, backfill if neccesary for new symbols
fetch hourly candles and append
recalculate TA stuff, save as new columns

Backfill:
start at today, minutes
It says max 1500, but default might be different. Adjust since so that the next fetch is 1500 (or length if errors) before the earliest entry in the response, and run again. Do this until page returns empty.
since = kucoin.milliseconds()
since = result[start] - 86400000
limit = 1440 (1d (the 86400000) difference expressed in number of intervals, minutes in this case.)
this will start from today, and try to fetch the previous days result, not allowing it to go over. Once the result is empty, we should convert to pandas, store, and continue to the next symbol.


To Do
-clean data (adjust for 0s and Nas)
-add basic statistics
-add technical indicators
-look into adding messari data for more exchanges, maybe alphavantage for stocks.


Ideas:
- Add different exchanges to understand correlation of different types of coins to getting listed on different exchanges.
- Find old list?

# Description of report builder
## Variable Controls
- Report Name
- Symbols
- Save Group
- Add Group
- Date Range
- Interval(s) (resample candles)

## Figure Variable Controls
- Figure Name
- Date Range
- Markup / WYSIWYG HTML editor
- Symbols
- Save Group
- Add Group
- Date Range
- Interval(s) (resample candles)
- Weighting (method or hard coded)
- Resample (frequency / function to return boolean)
- Axes / Plots 


In [ ]:

# mins_daily = 2000/30
# milli_sec_daily = mins_daily*60*1000
# delay = milli_sec_daily/2000


# for column in columns: 
#     query_params = {
#     'start': '2020-01-01',
#     'end': '2020-02-01',
#     'interval': '1d',
#     'columns': columns[column],
#     'order': 'ascending',
#     'format': 'json',
#     'timestamp-format': 'rfc3339'
#     }
#     historical = messari.get_asset_timeseries(asset_key='bitcoin', metric_id=column, **query_params)
#     m_data[column] = historical
# for metric in resp['data'][0]['metrics'].keys():
#     print(metric, resp['data'][0]['metrics'][metric] )

## View assets from messari, currently hardcoded to one month of BTC

In [25]:
metrics_dropown = widgets.Dropdown(
    options=sorted(all_metrics),
    description='Metric:',
    disabled=False
)
display(metrics_dropown)
view_metric_button = widgets.Button(description="View Metric")
output = widgets.Output()
current_metrics = {}

def view_metric(b):
    # if 'data' not in m_data[metrics_dropown.value]:
    query_params = {
    'start': '2020-01-01',
    'end': '2020-02-01',
    'interval': '1d',
    'columns': columns[metrics_dropown.value],
    'order': 'ascending',
    'format': 'json',
    'timestamp-format': 'rfc3339'
    }
    mess_collection = store.collection('Crypto.Metrics.{}'.format(metrics_dropown.value))

    historical = messari.get_asset_timeseries(asset_key='bitcoin', metric_id=metrics_dropown.value, **query_params)
    df = pd.DataFrame(historical['data']['values'])
    names = historical['data']['schema']['values_schema'].keys()
    column_index = {}
    i = 0
    for name in names:
        column_index[i] = name
        i += 1
    df = df.rename(columns=column_index)
    mess_collection.write('BTC', df)
    with output:
        print(df)

mining_metrics = [
    'hashrate',
    'diff.avg',
    'min.rev.usd',
    'min.rev.ntv',
    'blk.cnt',
    'blk.size.byte',
    'blk.size.bytes.avg',
]

transaction_metrics = [
    'fees',
    'fees.ntv', 
    'real.vol',
    'txn.vol',
    'txn.cnt',
    'txn.fee.med',
    'txn.fee.med.ntv',
    'txn.fee.avg',
    'txn.fee.avg.ntv',
    'txn.tsfr.cnt',
    'txn.tfr.val.ntv',
    'txn.tfr.erc20.cnt',
    'txn.tfr.erc721.cnt',
    'txn.tsfr.val.avg',
    'txn.tfr.val.med',
    'txn.tsfr.val.adj',
    'txn.tfr.val.adj.ntv',
    'act.addr.cnt',
]

exchange_metrics = [
    'exch.flow.in.usd',
    'exch.flow.out.usd',
    'exch.flow.out.usd.incl',
    'exch.flow.in.usd.incl',
    'exch.flow.in.ntv',
    'exch.flow.out.ntv',
    'exch.flow.out.ntv.incl',
    'exch.flow.in.ntv.incl',
    'exch.sply',
    'exch.sply.usd',
]

market_metrics = [
    'price',
    'mcap.out',
    'mcap.circ',
    'mcap.realized',
    'mcap.dom',
    'sply.total.iss',
    'sply.total.iss.ntv',
    'sply.liquid',
    'sply.circ',
    'sply.out',
    'new.iss.ntv',
    'new.iss.usd',
    'nvt.adj',
]

sentiment_metrics = [
    'reddit.subscribers',
    'reddit.active.users',
    ]
view_metric_button.on_click(view_metric)


display(view_metric_button, output)

Dropdown(description='Metric:', options=('act.addr.cnt', 'blk.cnt', 'blk.size.byte', 'blk.size.bytes.avg', 'da…

Button(description='View Metric', style=ButtonStyle())

Output()

In [48]:
all_assets_rn = messari.get_all_assets()['data']
for asset in all_assets_rn:
    # metadata
    messari_name = asset['name']
    messari_slug = asset['slug']
    messari_id = asset['id']
    symbol = asset['symbol']

    #labels, etc
    overview = asset['profile']['general']['overview']
    is_verified = overview['is_verified']
    tagline = overview['tagline']
    category = overview['category']
    sector = overview['sector']
    tags = overview['tags']
    project_details = overview['project_details']

    # organization, background, links
    background_details = asset['profile']['general']['background']['background_details']
    organization = asset['profile']['general']['background']['issuing_organizations']
    links = overview['official_links']

    # Roadmap
    roadmap = asset['profile']['general']['roadmap']

    # weird regulation thing
    regulation = asset['profile']['general']['regulation']

    #People and Orgs - gives people and orgs
    ind_contributors = asset['profile']['contributors']['individuals']
    org_contributors = asset['profile']['contributors']['organizations']
    ind_advisors = asset['profile']['advisors']['individuals']
    org_advisors = asset['profile']['advisors']['organizations']
    ind_investors = asset['profile']['investors']['individuals']
    org_investors = asset['profile']['investors']['organizations']

    #Ecosystem?

    ecosystem_assets = asset['profile']['ecosystem']['assets']
    ecosystem_orgs = asset['profile']['ecosystem']['organizations']

    # Economics

    token_details = asset['profile']['economics']['token']
    launch_details = asset['profile']['economics']['launch']
    consensus_details = asset['profile']['economics']['consensus_and_emission']
    treasury_details = asset['profile']['economics']['native_treasury']

    # Tecnology
    tech_details = asset['profile']['technology']['overview']['technology_details']
    client_repositories = asset['profile']['technology']['overview']['client_repositories']
    security_audits = asset['profile']['technology']['security']['audits']
    known_exploits = asset['profile']['technology']['security']['known_exploits_and_vulnerabilities']

    # Governance
    gov_details = asset['profile']['governance']['governance_details']
    gov_onchain = asset['profile']['governance']['onchain_governance']
    gov_grants = asset['profile']['governance']['grants']

    print(messari_name,  "\n")

Bitcoin 

Ethereum 

Tether 

BNB 

Cardano 

Dogecoin 

XRP 

Polkadot 

Uniswap 

Internet Computer 

Bitcoin Cash 

Litecoin 

Chainlink 

USD Coin 

Solana 

Theta Network 

VeChain 

Polygon 

Ethereum Classic 

Stellar 

